## Scikit learn to ONNX
- Neuromophic 연구 : ONNX <-> ML (Scikit learn)

- 로지스틱 회귀 모델 (Logistic Regression)

In [1]:
# Iris Data Load 
# Train a model.

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112,) (38,)


In [28]:
# Logistic Regression Fit
from sklearn.linear_model import LogisticRegression

# 모델 생성 및 학습
clr = LogisticRegression()
clr.fit(X_train, y_train)
clr

c:\users\urse\anaconda3\envs\untitled\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

# onnx 모델 지정
with open("logreg_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    

In [33]:
# Compute the prediction with ONNX Runtime

import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession("logreg_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [34]:
print(pred_onx)

[2 2 1 0 2 1 1 0 0 1 2 0 2 1 1 1 0 1 1 1 1 0 1 0 2 2 1 2 0 2 2 2 2 0 0 1 2
 2]


In [35]:
y_test

array([2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 2, 0, 2, 1, 2, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 2, 2])

In [36]:
# Accuracy 평가지표
from sklearn.metrics import accuracy_score

# Predict에 대한 Accuracy Score
print('accuracy_score', float(accuracy_score(y_test, pred_onx))) 

accuracy_score 0.9210526315789473
